# EuroPython 2015 Financial Aid RECIBI factory

Spreadsheet: https://docs.google.com/spreadsheets/d/1-T8o4Twh3OLvPX_Sv5L1-NeEyTuhYltsULicGFbB4YI/edit#gid=1784691601

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 99999;
//increase max size of output area

## Connection

In [ ]:
from   collections import OrderedDict

In [ ]:
import json
import gspread
from   oauth2client.client import SignedJwtAssertionCredentials

json_key = json.load(open('google_api_key.json'))
scope = ["https://spreadsheets.google.com/feeds"]
sheetdoc_key = "1--NeEyTuhYltsULicGFbB4YI"

In [ ]:
# authenticate
credentials = SignedJwtAssertionCredentials(json_key["client_email"], json_key["private_key"].encode("utf-8"), scope)
gc = gspread.authorize(credentials)

## Read data

In [ ]:
#Go to Google Sheets and share your spreadsheet with an email you have in your json_key['client_email']. 
#Otherwise you’ll get a SpreadsheetNotFound exception when trying to open it.
doc = gc.open_by_key(sheetdoc_key)
wks = doc.get_worksheet(1)

In [ ]:
def get_rows_data(wks, header='', start_row=1):
    all_rows = wks.get_all_values()
    if not header:
        header = all_rows[0]

    #print(list(zip(header, all_rows)))
    #[dict(zip(header, values)) for values in all_rows[1:]]
    return [dict(zip(header, values)) for values in all_rows[start_row:]]

In [ ]:
header = ['full_name', 'amount', 'amount_script', 'expense_docs', 'address']
data = get_rows_data(wks, header=header)

## Show information

In [ ]:
import pandas as pd
df = pd.DataFrame(data)
df

## Generate CSV file

In [ ]:
doclist_items = lambda row: '\n'.join(['\item {}'.format(doc) for doc in row['expense_docs'].split('\n')])
itemize_doclist = lambda row: '{}\n{}\n{}'.format('\\begin{enumerate}', doclist_items(row), '\\end{enumerate}')

df['expense_docs'] = df.apply(itemize_doclist, axis=1)

In [ ]:
responses_filepath = 'finaid_recibis.csv'
df.to_csv(responses_filepath)

In [ ]:
row_idx = 4

template_filepath  = 'acpyss_recibi_spa.tex'
fileid_field       = 'full_name'
row_id             = data[row_idx][fileid_field].replace(' ', '_')

In [ ]:
!docstamp -i $responses_filepath -t $template_filepath -f $fileid_field -c xelatex --idx $row_idx -v

In [ ]:
!open stamped/acpyss_recibi_spa.tex_FernandoMasanoriAshikaga.pdf